In [34]:
import pandas as pd

In [51]:
base = r"J:\Shared drives\TMD_TSA\Model\software\PopSim\popsim_nhri"

In [63]:
hh_in = base+r"\output\synthetic_households.csv"
per_in = base+r"\output\synthetic_persons.csv"
blck_taz = r"J:\Shared drives\TMD_TSA\Model\platform\inputs\zonal\shp\taz_2010block_assignment.csv"
hh_per_out = base+r"\output\PopulationSim_microhouseholds_2040_v20230203_RI.csv"

In [53]:
hh_df = pd.read_csv(hh_in)
per_df = pd.read_csv(per_in)

In [54]:
hh_df.head(), per_df.head()

(   household_id       PUMA     TAZ  HHNP  HHEMPCAT  HHINCPADJ
 0             1  2500100.0  6001.0     2         2      65107
 1             2  2500100.0  6001.0     4         2     176578
 2             3  2500100.0  6001.0     2         0      65699
 3             4  2500100.0  6001.0     4         1      69842
 4             5  2500100.0  6001.0     2         1      44391,
         PUMA     TAZ  household_id  EMP  AGEP  PINCPADJ
 0  2500100.0  6001.0             1    1    24     20716
 1  2500100.0  6001.0             1    1    26     44391
 2  2500100.0  6001.0             2    1    51    100620
 3  2500100.0  6001.0             2    1    52     75958
 4  2500100.0  6001.0             2    0    15         0)

### Post Processing

1. Update attribute names
3. Calculate household attributes of workers, persons, and children
4. Attach household attributes to person table
5. zero out personal income for non-workers (and all negative incomes)
6. Assign a representative block
2. Create unique houseold and person ID


In [55]:
#Update attribute names
hh_df = hh_df.rename(columns={"household_id":"hid","HHINCPADJ":"hh_inc","TAZ":"taz_id"})
per_df = per_df.rename(columns={"household_id":"hid","PINCPADJ":"wage_inc","EMP":'is_worker',"AGEP":"age"})

# drop duplicate attributes
per_df = per_df.drop(columns={"PUMA","TAZ"})

hh_df, per_df

(             hid       PUMA    taz_id  HHNP  HHEMPCAT  hh_inc
 0              1  2500100.0    6001.0     2         2   65107
 1              2  2500100.0    6001.0     4         2  176578
 2              3  2500100.0    6001.0     2         0   65699
 3              4  2500100.0    6001.0     4         1   69842
 4              5  2500100.0    6001.0     2         1   44391
 ...          ...        ...       ...   ...       ...     ...
 3945121  3945122  4400400.0  208992.0     1         1   27668
 3945122  3945123  4400400.0  208992.0     1         1   27668
 3945123  3945124  4400400.0  208992.0     3         2   31357
 3945124  3945125  4400400.0  208992.0     2         1   27668
 3945125  3945126  4400400.0  208992.0     2         2    4353
 
 [3945126 rows x 6 columns],
              hid  is_worker  age  wage_inc
 0              1          1   24     20716
 1              1          1   26     44391
 2              2          1   51    100620
 3              2          1   52    

In [56]:
#Calculate household attributes of workers, persons, and children

hh_per_df = pd.DataFrame(
        per_df.groupby('hid')['hid'].count()
        ).rename(columns={"hid":"persons"})

hh_chld_df = pd.DataFrame(
        per_df[per_df['age']<18].groupby('hid')['hid'].count()
        ).rename(columns={"hid":"children"})

hh_wrk_df = pd.DataFrame(
        per_df[per_df['is_worker']==1].groupby('hid')['hid'].count()
        ).rename(columns={"hid":"workers"})

hh_df = hh_df.merge(
    hh_per_df, on="hid", how="left").merge(
    hh_chld_df, on="hid", how="left").merge(
    hh_wrk_df, on="hid", how="left").fillna(0)

hh_df

#hh_per_df, hh_chld_df, hh_wrk_df

hid       PUMA    taz_id  HHNP  HHEMPCAT  hh_inc  persons  \
0              1  2500100.0    6001.0     2         2   65107        2   
1              2  2500100.0    6001.0     4         2  176578        4   
2              3  2500100.0    6001.0     2         0   65699        2   
3              4  2500100.0    6001.0     4         1   69842        4   
4              5  2500100.0    6001.0     2         1   44391        2   
...          ...        ...       ...   ...       ...     ...      ...   
3945121  3945122  4400400.0  208992.0     1         1   27668        1   
3945122  3945123  4400400.0  208992.0     1         1   27668        1   
3945123  3945124  4400400.0  208992.0     3         2   31357        3   
3945124  3945125  4400400.0  208992.0     2         1   27668        2   
3945125  3945126  4400400.0  208992.0     2         2    4353        2   

         children  workers  
0             0.0      2.0  
1             2.0      2.0  
2             0.0      0.0  
3             2.0      1.0  
4             0.0      1.0  
...           ...      ...  
3945121       0.0      1.0  
3945122       0.0      1.0  
3945123       1.0      2.0  
3945124       1.0      1.0  
3945125       0.0      2.0  

[3945126 rows x 9 columns]

In [57]:
#Attach household attributes to person table
hh_per_df = per_df.merge(hh_df, on="hid")
hh_per_df

hid  is_worker  age  wage_inc       PUMA    taz_id  HHNP  \
0              1          1   24     20716  2500100.0    6001.0     2   
1              1          1   26     44391  2500100.0    6001.0     2   
2              2          1   51    100620  2500100.0    6001.0     4   
3              2          1   52     75958  2500100.0    6001.0     4   
4              2          0   15         0  2500100.0    6001.0     4   
...          ...        ...  ...       ...        ...       ...   ...   
8857897  3945124          0   10         0  4400400.0  208992.0     3   
8857898  3945125          1   41     27668  4400400.0  208992.0     2   
8857899  3945125          0   15         0  4400400.0  208992.0     2   
8857900  3945126          1   20      3689  4400400.0  208992.0     2   
8857901  3945126          1   21       664  4400400.0  208992.0     2   

         HHEMPCAT  hh_inc  persons  children  workers  
0               2   65107        2       0.0      2.0  
1               2   65107        2       0.0      2.0  
2               2  176578        4       2.0      2.0  
3               2  176578        4       2.0      2.0  
4               2  176578        4       2.0      2.0  
...           ...     ...      ...       ...      ...  
8857897         2   31357        3       1.0      2.0  
8857898         1   27668        2       1.0      1.0  
8857899         1   27668        2       1.0      1.0  
8857900         2    4353        2       0.0      2.0  
8857901         2    4353        2       0.0      2.0  

[8857902 rows x 12 columns]

In [42]:
#zero out personal income for non-workers (and all negative incomes)
hh_per_df.loc[hh_per_df['is_worker']==0,'wage_inc']=0
hh_per_df.loc[hh_per_df['wage_inc']<0,'wage_inc']=0
hh_per_df['wage_inc'].describe()

count    7.882446e+06
mean     3.075259e+04
std      5.831827e+04
min      0.000000e+00
25%      0.000000e+00
50%      1.883000e+03
75%      4.519100e+04
max      1.424434e+06
Name: wage_inc, dtype: float64

In [58]:
#Assign a representative block
blk_lu = pd.read_csv(blck_taz)
taz_lu = blk_lu.drop_duplicates(subset="taz_id")

hh_per_df = hh_per_df.merge(taz_lu,on="taz_id", how="left")
hh_per_df

hid  is_worker  age  wage_inc       PUMA    taz_id  HHNP  \
0              1          1   24     20716  2500100.0    6001.0     2   
1              1          1   26     44391  2500100.0    6001.0     2   
2              2          1   51    100620  2500100.0    6001.0     4   
3              2          1   52     75958  2500100.0    6001.0     4   
4              2          0   15         0  2500100.0    6001.0     4   
...          ...        ...  ...       ...        ...       ...   ...   
8857897  3945124          0   10         0  4400400.0  208992.0     3   
8857898  3945125          1   41     27668  4400400.0  208992.0     2   
8857899  3945125          0   15         0  4400400.0  208992.0     2   
8857900  3945126          1   20      3689  4400400.0  208992.0     2   
8857901  3945126          1   21       664  4400400.0  208992.0     2   

         HHEMPCAT  hh_inc  persons  children  workers         block_id  
0               2   65107        2       0.0      2.0  250039241001000  
1               2   65107        2       0.0      2.0  250039241001000  
2               2  176578        4       2.0      2.0  250039241001000  
3               2  176578        4       2.0      2.0  250039241001000  
4               2  176578        4       2.0      2.0  250039241001000  
...           ...     ...      ...       ...      ...              ...  
8857897         2   31357        3       1.0      2.0           208992  
8857898         1   27668        2       1.0      1.0           208992  
8857899         1   27668        2       1.0      1.0           208992  
8857900         2    4353        2       0.0      2.0           208992  
8857901         2    4353        2       0.0      2.0           208992  

[8857902 rows x 13 columns]

In [59]:
#Create unique houseold and person ID
hh_per_df['hid'] = hh_per_df['block_id'].astype(str) + "_" + hh_per_df['hid'].astype(str)
hh_per_df['person_num'] = hh_per_df.groupby('hid').cumcount() + 1
hh_per_df

hid  is_worker  age  wage_inc       PUMA    taz_id  \
0        250039241001000_1          1   24     20716  2500100.0    6001.0   
1        250039241001000_1          1   26     44391  2500100.0    6001.0   
2        250039241001000_2          1   51    100620  2500100.0    6001.0   
3        250039241001000_2          1   52     75958  2500100.0    6001.0   
4        250039241001000_2          0   15         0  2500100.0    6001.0   
...                    ...        ...  ...       ...        ...       ...   
8857897     208992_3945124          0   10         0  4400400.0  208992.0   
8857898     208992_3945125          1   41     27668  4400400.0  208992.0   
8857899     208992_3945125          0   15         0  4400400.0  208992.0   
8857900     208992_3945126          1   20      3689  4400400.0  208992.0   
8857901     208992_3945126          1   21       664  4400400.0  208992.0   

         HHNP  HHEMPCAT  hh_inc  persons  children  workers         block_id  \
0           2         2   65107        2       0.0      2.0  250039241001000   
1           2         2   65107        2       0.0      2.0  250039241001000   
2           4         2  176578        4       2.0      2.0  250039241001000   
3           4         2  176578        4       2.0      2.0  250039241001000   
4           4         2  176578        4       2.0      2.0  250039241001000   
...       ...       ...     ...      ...       ...      ...              ...   
8857897     3         2   31357        3       1.0      2.0           208992   
8857898     2         1   27668        2       1.0      1.0           208992   
8857899     2         1   27668        2       1.0      1.0           208992   
8857900     2         2    4353        2       0.0      2.0           208992   
8857901     2         2    4353        2       0.0      2.0           208992   

         person_num  
0                 1  
1                 2  
2                 1  
3                 2  
4                 3  
...             ...  
8857897           3  
8857898           1  
8857899           2  
8857900           1  
8857901           2  

[8857902 rows x 14 columns]

In [33]:
# clean and export
hh_per_df[['block_id',
           'hid',
           'hh_inc',
           'persons',
           'workers',
           'children',
           'person_num',
           'age',
           'is_worker',
           'wage_inc']].to_csv(hh_per_out, index=False)